# MobileNet

## Questions:
- **What are the advantages and tradeoffs of the MobileNet family of CNNs?**
    - MobileNet family of CNNs are models with significantly fewer parameters and smaller disk space when compared with other models such VGG16. MobileNet models requires smaller hardware requirements during training, using GPU is not optional. The tradeoff would be its accuracy. Since they have fewer parameters, the model may show results with  a reduced accuracy due to its limited weights. 

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print(f"Num GPUs Available: {len(physical_devices)}")

Num GPUs Available: 1


In [10]:
mobile = tf.keras.applications.mobilenet.MobileNet()

17225924/17225924 [==============================] - 1s 0us/step


In [11]:
def prepare_image(file):
    img_path = "/kaggle/input/sign-language-digits/Sign-Language-Digits-Dataset/"
    img = image.load(img_path + file, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
# from IPython.display import Image
# Image()